<a href="https://colab.research.google.com/github/ananyadix/chatbot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import json

data = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": ["Hi", "Hello", "How are you?", "Is anyone there?", "Good day"],
            "responses": ["Hello!", "Hi there!", "Greetings!", "How can I assist you?"]
        },
        {
            "tag": "goodbye",
            "patterns": ["Bye", "See you later", "Goodbye"],
            "responses": ["Goodbye!", "See you later!", "Have a great day!"]
        },
        {
            "tag": "thanks",
            "patterns": ["Thanks", "Thank you", "That's helpful"],
            "responses": ["You're welcome!", "Happy to help!", "My pleasure!"]
        },
        {
            "tag": "noanswer",
            "patterns": [],
            "responses": ["Sorry, I don't understand that.", "Can you please rephrase?", "I'm not sure how to respond to that."]
        },
        {
            "tag": "options",
            "patterns": ["What can you do?", "How can you help?", "What services do you offer?"],
            "responses": ["I can assist you with a variety of tasks such as answering questions, providing information, and more!"]
        },
        {
            "tag": "age",
            "patterns": ["How old are you?", "What is your age?"],
            "responses": ["I'm a computer program, so I don't have an age.", "I was created in 2024."]
        },
        {
            "tag": "name",
            "patterns": ["What is your name?", "Who are you?", "Can you tell me your name?"],
            "responses": ["I am a chatbot created by a little programmer.", "You can call me Coco."]
        },
        {
            "tag": "weather",
            "patterns": ["What's the weather like?", "Is it going to rain today?", "How's the weather?"],
            "responses": ["I can't check the weather right now, but you can use a weather service or app to find out."]
        }
    ]
}

with open('training_data.json', 'w') as outfile:
    json.dump(data, outfile, indent=4)

In [35]:
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from keras.models import load_model
from nltk.tokenize import word_tokenize
import tensorflow as tf

In [8]:
lemmatizer=WordNetLemmatizer()

In [9]:
with open('/content/training_data.json') as file:
    intents = json.load(file)

In [10]:
words=[]
classes=[]
documents=[]
ignore_letters=['?','!','.',',']

In [14]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        wordList = nltk.word_tokenize(pattern)
        words.extend(wordList)
        documents.append((wordList, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [16]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [18]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

In [19]:
classes=sorted(set(classes))

In [20]:
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [21]:
training=[]
outputEmpty=[0]*len(classes)

In [22]:
print(classes)

['age', 'goodbye', 'greeting', 'name', 'options', 'thanks', 'weather']


In [23]:
print(documents)

[(['Hi'], 'greeting'), (['Hello'], 'greeting'), (['How', 'are', 'you', '?'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['What', 'can', 'you', 'do', '?'], 'options'), (['How', 'can', 'you', 'help', '?'], 'options'), (['What', 'services', 'do', 'you', 'offer', '?'], 'options'), (['How', 'old', 'are', 'you', '?'], 'age'), (['What', 'is', 'your', 'age', '?'], 'age'), (['What', 'is', 'your', 'name', '?'], 'name'), (['Who', 'are', 'you', '?'], 'name'), (['Can', 'you', 'tell', 'me', 'your', 'name', '?'], 'name'), (['What', "'s", 'the', 'weather', 'like', '?'], 'weather'), (['Is', 'it', 'going', 'to', 'rain', 'today', '?'], 'weather'), (['How', "'s", 'the', 'weather', '?'], 'weather')]


In [24]:
print(words)

["'s", 'Bye', 'Can', 'Good', 'Goodbye', 'Hello', 'Hi', 'How', 'Is', 'See', 'Thank', 'Thanks', 'That', 'What', 'Who', 'age', 'anyone', 'are', 'can', 'day', 'do', 'going', 'help', 'helpful', 'is', 'it', 'later', 'like', 'me', 'name', 'offer', 'old', 'rain', 'service', 'tell', 'the', 'there', 'to', 'today', 'weather', 'you', 'your']


In [26]:
for document in documents:
  bag=[]
  word_pattern=document[0]
  wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in word_pattern]
  for word in words:
    bag.append(1) if word in word_pattern else bag.append(0)
  output_row=list(outputEmpty)
  output_row[classes.index(document[1])]=1
  training.append([bag,output_row])

In [30]:
random.shuffle(training)
training = np.array(training, dtype=object)

In [32]:
print(training)

[[list([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
  list([0, 1, 0, 0, 0, 0, 0])]
 [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])
  list([0, 1, 0, 0, 0, 0, 0])]
 [list([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])
  list([0, 0, 0, 0, 1, 0, 0])]
 [list([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
  list([0, 0, 1, 0, 0, 0, 0])]
 [list([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
  list([0, 1, 0, 0, 0, 0, 0])]
 [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
  list([0, 0, 0, 1, 0, 0, 0])]
 [li

In [53]:
trainX = np.array(list(training[:, 0]), dtype=np.float32)
trainY = np.array(list(training[:, 1]), dtype=np.float32)

In [54]:
print('trainX shape:', trainX.shape)
print('trainY shape:', trainY.shape)

trainX shape: (22, 42)
trainY shape: (22, 7)


In [56]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape=(len(trainX[1]),), activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(trainY[1]), activation='softmax'))

In [57]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [58]:
hist = model.fit(trainX, trainY, epochs=200, batch_size=5, verbose=1)

Epoch 1/200
5/5 [==============================] - 1s 6ms/step - loss: 2.0689 - accuracy: 0.2273
Epoch 2/200
5/5 [==============================] - 0s 5ms/step - loss: 2.0503 - accuracy: 0.1818
Epoch 3/200
5/5 [==============================] - 0s 5ms/step - loss: 1.8897 - accuracy: 0.1818
Epoch 4/200
5/5 [==============================] - 0s 5ms/step - loss: 1.8733 - accuracy: 0.1818
Epoch 5/200
5/5 [==============================] - 0s 4ms/step - loss: 1.9376 - accuracy: 0.1364
Epoch 6/200
5/5 [==============================] - 0s 5ms/step - loss: 1.9419 - accuracy: 0.2727
Epoch 7/200
5/5 [==============================] - 0s 6ms/step - loss: 1.7691 - accuracy: 0.3182
Epoch 8/200
5/5 [==============================] - 0s 5ms/step - loss: 1.7622 - accuracy: 0.3636
Epoch 9/200
5/5 [==============================] - 0s 4ms/step - loss: 1.7657 - accuracy: 0.2727
Epoch 10/200
5/5 [==============================] - 0s 4ms/step - loss: 1.7129 - accuracy: 0.3636
Epoch 11/200
5/5 [===========

In [59]:
model.save('chatbot_model.h5')
print('Done')

Done


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [60]:
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbot_model.h5')

In [61]:
def clean_up_sen(sentence):
  sentence_word=nltk.word_tokenize(sentence)
  sentence_word=[lemmatizer.lemmatize(word) for word in sentence_word]
  return sentence_word